# 웹 검색 (Web Search)
웹 검색 기능은 모델이 **인터넷의 최신 정보를 실시간으로 조회**하고, **출처가 명시된 답변(citations 포함)**을 제공할 수 있도록 합니다.
Responses API를 사용하면 API 요청의 tools 배열에서 웹 검색을 구성하여 콘텐츠를 생성함으로써 웹 검색을 활성화할 수 있습니다. 다른 도구와 마찬가지로, 모델은 입력 프롬프트의 내용에 따라 웹 검색 여부를 선택할 수 있습니다.  

### OpenAI 모델이 제공하는 웹 검색의 세 가지 주요 유형

1. **비-추론형 웹 검색 (Non-reasoning web search)**

    * 모델이 사용자의 쿼리를 웹 검색 도구에 직접 전달합니다.
    * 도구는 상위 검색 결과를 기반으로 응답을 반환합니다.
    * 모델 내부에서 별도의 **계획(plan)** 이나 **추론(reasoning)** 은 수행하지 않습니다.
    * **장점:** 매우 빠르고 단순한 조회(lookup)에 적합
    * **예시:** “오늘 서울 날씨” 같은 간단한 질의

2. **에이전트형 검색 (Agentic search with reasoning models)**

    * **추론(reasoning)** 이 가능한 모델이 **검색 과정을 스스로 관리**합니다.
    * 모델은 검색을 체인 오브 소트(chain of thought) 내 일부로 수행하며,
      결과를 분석하고 **추가 검색이 필요한지 여부를 판단**합니다.
    * **특징:** 유연하고 복잡한 워크플로에 적합하지만, 단순 검색보다 시간이 더 걸림
    * **예시:** GPT-5의 reasoning level을 조정하여 검색의 깊이(정밀도)와 속도(지연시간)를 함께 조절 가능

3. **심층 연구 (Deep research)**

    * **에이전트 주도(agent-driven)**의 고급 방식으로,
      **깊이 있고 장시간 진행되는 탐색형 조사**를 수행합니다.
    * 모델은 수백 개의 출처를 활용하며, chain of thought의 일부로 다단계 검색을 실행합니다.
    * **특징:** 몇 분간 실행될 수 있으며, **백그라운드 모드**에서 사용하는 것이 권장됨
    * **주로 사용하는 모델:** `o3-deep-research`, `o4-mini-deep-research`, 또는 **GPT-5 (reasoning level=high)**

모델 응답을 생성할 때, 내장 도구와 원격 MCP 서버를 사용하여 기능을 확장할 수 있습니다. 이를 통해 모델이 웹 검색, 파일 검색, 사용자 함수 호출, 또는 외부 서비스 접근이 가능해집니다.

## Web search 

* API 요청 시 `tools` 배열 안에 **web search 도구(web)** 를 추가합니다.
* 모델은 **입력 프롬프트의 내용에 따라**, 필요 시 자동으로 웹 검색을 수행하거나 생략할 수 있습니다.

## 도메인 필터링
웹 검색의 도메인 필터링을 사용하면 검색 결과를 특정 도메인 집합으로 제한할 수 있습니다.  
filters 매개변수를 사용하여 최대 20개의 URL 허용 목록을 설정할 수 있습니다.   
URL 형식을 지정할 때는 HTTP 또는 HTTPS 접두사를 생략하세요. 예를 들어 https://openai.com/ 대신 openai.com을 사용합니다. 이 방식은 하위 도메인도 검색에 포함시킵니다.   
도메인 필터링은 web_search 도구를 사용하는 Responses API에서만 사용할 수 있습니다. 

### Sources
웹 검색 중에 조회된 모든 URL을 보려면 sources 필드를 사용하세요.   
가장 관련성 높은 참조만 보여주는 인라인 인용과 달리, sources는 모델이 응답을 작성할 때 참조한 전체 URL 목록을 반환합니다.   
출처의 수는 종종 인용의 수보다 많습니다. 실시간 제3자 피드도 여기에 표시되며 oai-sports, oai-weather, oai-finance로 라벨이 지정됩니다.   
sources 필드는 web_search 및 web_search_preview 도구 모두에서 사용할 수 있습니다.

In [ ]:
                                     # 복잡한 논리나 분석이 필요 없는 간단 검색이라 "low"로 지정
                                     # (필요시 web_search 실행, 불필요시 내부 지식만 사용)
                                     # 응답 결과에 검색 출처(링크/도메인) 정보를 포함하도록 설정
                                     # 출력 텍스트에는 표시되지 않지만 response 객체에 포함됨
# 모델의 텍스트 응답 출력

### 사용자 위치  
지역을 기준으로 검색 결과를 좁히려면 국가, 도시, 지역 및/또는 시간대를 사용하여 대략적인 사용자 위치를 지정할 수 있습니다.  
도시 및 지역 필드는 각각 미니애폴리스와 미네소타처럼 자유 텍스트 문자열입니다.  
국가 필드는 US처럼 두 글자 ISO 국가 코드입니다.

| `type` 값        | 의미                         | 예시                   |
| --------------- | -------------------------- | -------------------- |
| `"approximate"` | **대략적 위치** (도시·국가 수준)      | 서울, 대한민국 정도까지만 알려줌   |
| `"precise"`     | **정밀 위치** (좌표 또는 상세 주소 수준) | 위도·경도, 특정 지점 근처까지 포함 |
| `"none"`        | 위치 정보 제공 안 함               | 전 세계 범위로 검색          |